In [1]:
import logging
import scrapy
import json
import re
import operator
    
from scrapy.crawler import CrawlerProcess

<b>Задание 1</b>

In [2]:
class TopLinksSpider(scrapy.Spider):
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'FEED_FORMAT':'json',
        'FEED_URI': 'wikipedia.json'
    }
    
    name = 'all_links'
    start_urls = [
        'https://en.wikipedia.org/wiki/Information_retrieval', 
        'https://en.wikipedia.org/wiki/Principality_of_Sealand',
        'https://en.wikipedia.org/wiki/Belarus',
        'https://en.wikipedia.org/wiki/Penguin',
        'https://en.wikipedia.org/wiki/Robert_Rodriguez'
    ]
    
    header_selector = 'h1#firstHeading.firstHeading *::text'
    snippet_selector = 'string(//div[@id="mw-content-text"]//p[position() = 1])'
    body_link_selector = '(//div[@id="mw-content-text"]//*//a/@href)[position() < 100]'
    allowed_re = re.compile('https://en\.wikipedia\.org/wiki/'
                            '(?!((File|Talk|Category|Portal|Special|Wikipedia'
                            '|Help|Draft):|Main_Page)).+')
    
    visited_urls = set()
    visited_urls_count = len(start_urls)
    limit = 10000
       
    def parse(self, response):
        self.visited_urls.add(response.url)

        links = response.xpath(self.body_link_selector).extract()
        
        valid_page_urls = set()
        for url in self.filter_invalid_urls(response, links):
            if url in valid_page_urls:
                continue
                
            if not url in self.visited_urls and self.visited_urls_count < self.limit:
                valid_page_urls.add(url)
                
                self.visited_urls_count+=1
                self.visited_urls.add(url)
                
                yield scrapy.Request(url, callback=self.parse)
            elif url in self.visited_urls:
                valid_page_urls.add(url)
                
                    
        yield {
            'url': response.url,
            'title': response.css(self.header_selector).extract_first(),
            'snippet': response.xpath(self.snippet_selector).extract_first()[:255],
            'links': valid_page_urls
        }
                    
                    
    def filter_invalid_urls(self, response, links):
        for link in links:
            if '#' in link:
                continue
                
            url = response.urljoin(link)
            
            if self.allowed_re.match(url):
                yield url
                
process = CrawlerProcess({'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'})

process.crawl(TopLinksSpider)
process.start()

2018-04-12 23:11:30 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: scrapybot)
2018-04-12 23:11:30 [scrapy.utils.log] INFO: Versions: lxml 3.7.3.0, libxml2 2.9.4, cssselect 1.0.3, parsel 1.4.0, w3lib 1.19.0, Twisted 17.9.0, Python 3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)], pyOpenSSL 17.0.0 (OpenSSL 1.0.2m  2 Nov 2017), cryptography 1.8.1, Platform Windows-10-10.0.16299-SP0
2018-04-12 23:11:30 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'wikipedia.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


<b>Задание 2</b>

In [2]:
def load_data(file_name):
    with open(file_name, 'r') as file:    
        return json.load(file)

# reading loaded Wikipedia pages
wikipedia_pages = load_data('wikipedia.json')

In [3]:
import networkx as nx
    
def build_graph(pages):
    graph = nx.DiGraph()
    
    for page in pages:
        url = page["url"]
        title = page["title"]
        snippet = page["snippet"]
        graph.add_node(url, title=title, snippet=snippet)
        
        
    for page in pages:
        url = page["url"]
        links = page["links"]
        
        #due to the synchronization issues some 'invali' links are still downloaded so we want to check if target node exists
        graph.add_edges_from([(url, link) for link in links if graph.has_node(link)]) 
    
    return graph

In [4]:
# Строим граф
wikipedia_graph = build_graph(wikipedia_pages)

<b>Задание 3</b>

In [6]:
pagerank_output_format = """
Link: {}
Rank: {}
Input edges: {}
Output edges: {}
Title: {}
Snippet: {}
"""

def print_top_10(graph, ranked_pages, output_format):
    top_10 = sorted(ranked_pages.items(), key=operator.itemgetter(1), reverse=True)[:10]
    for url, rank in top_10:
        node = graph.node[url]
        result = output_format.format(url, rank, len(graph.in_edges(url)), len(graph.out_edges(url)), node["title"], node["snippet"])
        print(result)

In [11]:
ranked_pages = nx.pagerank(wikipedia_graph)
print_top_10(wikipedia_graph, ranked_pages, pagerank_output_format)


Link: https://en.wikipedia.org/wiki/United_States
Rank: 0.005666386556743823
Input edges: 839
Output edges: 32
Title: United States
Snippet: Coordinates: 40°N 100°W﻿ / ﻿40°N 100°W﻿ / 40; -100


Link: https://en.wikipedia.org/wiki/International_Standard_Book_Number
Rank: 0.003926914819739761
Input edges: 949
Output edges: 12
Title: International Standard Book Number
Snippet: The International Standard Book Number (ISBN) is a unique[a][b] numeric commercial book identifier. Publishers purchase ISBNs from an affiliate of the International ISBN Agency.[1]


Link: https://en.wikipedia.org/wiki/United_Kingdom
Rank: 0.003824170480641125
Input edges: 446
Output edges: 38
Title: United Kingdom
Snippet: – in Europe  (green & dark grey)
– in the European Union  (green)


Link: https://en.wikipedia.org/wiki/Demonym
Rank: 0.0036270451028315998
Input edges: 320
Output edges: 9
Title: Demonym
Snippet: A demonym (/ˈdɛmənɪm/; δῆμος dẽmos "people, tribe", ὄόνομα ónoma "name") is a word that identifies 

<b>Задание 4</b>

In [12]:
ranked_pages = nx.pagerank(wikipedia_graph, alpha=0.3)
print_top_10(wikipedia_graph, ranked_pages, pagerank_output_format)


Link: https://en.wikipedia.org/wiki/International_Standard_Book_Number
Rank: 0.002770106130693042
Input edges: 949
Output edges: 12
Title: International Standard Book Number
Snippet: The International Standard Book Number (ISBN) is a unique[a][b] numeric commercial book identifier. Publishers purchase ISBNs from an affiliate of the International ISBN Agency.[1]


Link: https://en.wikipedia.org/wiki/United_States
Rank: 0.0024421214021640137
Input edges: 839
Output edges: 32
Title: United States
Snippet: Coordinates: 40°N 100°W﻿ / ﻿40°N 100°W﻿ / 40; -100


Link: https://en.wikipedia.org/wiki/Geographic_coordinate_system
Rank: 0.001685224237248412
Input edges: 557
Output edges: 9
Title: Geographic coordinate system
Snippet: A geographic coordinate system is a coordinate system used in geography that enables every location on Earth to be specified by a set of numbers, letters or symbols.[n 1] The coordinates are often chosen such that one of the numbers represents a vertical 


Link: http

In [13]:
ranked_pages = nx.pagerank(wikipedia_graph, alpha=0.5)
print_top_10(wikipedia_graph, ranked_pages, pagerank_output_format)


Link: https://en.wikipedia.org/wiki/United_States
Rank: 0.003911384813617977
Input edges: 839
Output edges: 32
Title: United States
Snippet: Coordinates: 40°N 100°W﻿ / ﻿40°N 100°W﻿ / 40; -100


Link: https://en.wikipedia.org/wiki/International_Standard_Book_Number
Rank: 0.003867354511600385
Input edges: 949
Output edges: 12
Title: International Standard Book Number
Snippet: The International Standard Book Number (ISBN) is a unique[a][b] numeric commercial book identifier. Publishers purchase ISBNs from an affiliate of the International ISBN Agency.[1]


Link: https://en.wikipedia.org/wiki/Geographic_coordinate_system
Rank: 0.0025816652981343383
Input edges: 557
Output edges: 9
Title: Geographic coordinate system
Snippet: A geographic coordinate system is a coordinate system used in geography that enables every location on Earth to be specified by a set of numbers, letters or symbols.[n 1] The coordinates are often chosen such that one of the numbers represents a vertical 


Link: http

In [51]:
ranked_pages = nx.pagerank(wikipedia_graph, alpha=0.95)
print_top_10(wikipedia_graph, ranked_pages, pagerank_output_format)


Link: https://en.wikipedia.org/wiki/United_States
Rank: 0.005348731816264987
Input edges: 839
Output edges: 32
Title: United States
Snippet: Coordinates: 40°N 100°W﻿ / ﻿40°N 100°W﻿ / 40; -100


Link: https://en.wikipedia.org/wiki/United_Kingdom
Rank: 0.004482406389023244
Input edges: 446
Output edges: 38
Title: United Kingdom
Snippet: – in Europe  (green & dark grey)
– in the European Union  (green)


Link: https://en.wikipedia.org/wiki/Demonym
Rank: 0.004227088911323904
Input edges: 320
Output edges: 9
Title: Demonym
Snippet: A demonym (/ˈdɛmənɪm/; δῆμος dẽmos "people, tribe", ὄόνομα ónoma "name") is a word that identifies residents or natives of a particular place, which is derived from the name of that particular place.[1]


Link: https://en.wikipedia.org/wiki/Market_(economics)
Rank: 0.003610739134934194
Input edges: 40
Output edges: 13
Title: Market (economics)
Snippet: A market is one of the many varieties of systems, institutions, procedures, law reinforcement and infrastructur

In [7]:
ranked_pages = nx.pagerank(wikipedia_graph, alpha=1)
print_top_10(wikipedia_graph, ranked_pages, pagerank_output_format)


Link: https://en.wikipedia.org/wiki/Market_(economics)
Rank: 0.0071259642367956875
Input edges: 40
Output edges: 13
Title: Market (economics)
Snippet: A market is one of the many varieties of systems, institutions, procedures, law reinforcement and infrastructures whereby parties engage in exchange. While parties may exchange goods and services by barter, most markets rely on sellers offering their good


Link: https://en.wikipedia.org/wiki/Aluminium
Rank: 0.006659976245798094
Input edges: 32
Output edges: 1
Title: Aluminium
Snippet: Aluminium or aluminum is a chemical element with symbol Al and atomic number 13. It is a silvery-white, soft, nonmagnetic and ductile metal in the boron group. By mass, aluminium makes up about 8% of the Earth's crust; it is the third most abundant elemen


Link: https://en.wikipedia.org/wiki/Chromium
Rank: 0.006438003416294344
Input edges: 8
Output edges: 1
Title: Chromium
Snippet: Chromium is a chemical element with symbol Cr and atomic number 24. It is

Похоже, что топ во всех трёх случаях примерно на половину состоит из одних и тех же ссылок. Ссылка на США присутствует везде в ТОП-2.

Кажется, что с ростом alpha должен расти вклад в PR входящих ссылок. Я поглядел также вот такую статью: http://vigna.di.unimi.it/ftp/papers/PageRankAsFunction.pdf
И кажется, что с ростои alpha растёт PageRank страниц, которые входят в некие циклы. Собственно кажется, что это выражается в том, что при alpha == 0.95 у нас уже в top-5 есть US, UK и English. В то же время при прочих значениях данные ссылки тоже есть, но они более распределены по выборке. Я попробовал alpha = 1 и там вообще сплошные термины из экономики. Часть из них также присуствовала и при прочих alpha, но часть абсолютно новые. При этом про US уже ничего нет.

Кажется, что да при большом alpha возрастает роль циклов и как следствие падает разнообразие выборки результатов. При меньших alpha разнообразие результатов куда большее.

<b>Задание 5</b>

In [17]:
hubs, authorities = nx.hits(wikipedia_graph, max_iter=500)

In [18]:
hubs_output_format = """
Link: {}
Hub value: {}
Input edges: {}
Output edges: {}
Title: {}
Snippet: {}
"""

print_top_10(wikipedia_graph, hubs, hubs_output_format)


Link: https://en.wikipedia.org/wiki/List_of_American_films_of_1975
Hub value: 0.012897954584216604
Input edges: 62
Output edges: 86
Title: List of American films of 1975
Snippet: A list of American films released in 1975.


Link: https://en.wikipedia.org/wiki/List_of_American_films_of_1974
Hub value: 0.0128959714484279
Input edges: 63
Output edges: 86
Title: List of American films of 1974
Snippet: A list of American films released in 1974.


Link: https://en.wikipedia.org/wiki/List_of_American_films_of_1973
Hub value: 0.012890080612141544
Input edges: 66
Output edges: 86
Title: List of American films of 1973
Snippet: A list of American films released in 1973.
The Sting won the Academy Award for Best Picture. The highest-grossing film of the year was The Exorcist.


Link: https://en.wikipedia.org/wiki/List_of_American_films_of_1972
Hub value: 0.0128881373375523
Input edges: 67
Output edges: 86
Title: List of American films of 1972
Snippet: A list of American films released in 1972.


L

In [23]:
authorities_output_format = """
Link: {}
Authority value: {}
Input edges: {}
Output edges: {}
Title: {}
Snippet: {}
"""

print_top_10(wikipedia_graph, authorities, authorities_output_format)


Link: https://en.wikipedia.org/wiki/1910s_in_film
Authority value: 0.011870326540923305
Input edges: 101
Output edges: 3
Title: 1910s in film
Snippet: The decade of the 1910s in film involved some significant films.


Link: https://en.wikipedia.org/wiki/1900s_in_film
Authority value: 0.011870326540923305
Input edges: 101
Output edges: 3
Title: 1900s in film
Snippet: The decade of the 1900s in film involved some significant films.


Link: https://en.wikipedia.org/wiki/1890s_in_film
Authority value: 0.011870326540923305
Input edges: 101
Output edges: 3
Title: 1890s in film
Snippet: The decade of the 1890s in film involved some significant events.


Link: https://en.wikipedia.org/wiki/1920s_in_film
Authority value: 0.01183680689048576
Input edges: 95
Output edges: 3
Title: 1920s in film
Snippet: The decade of the 1920s in film involved many significant films.


Link: https://en.wikipedia.org/wiki/Cinema_of_the_United_States
Authority value: 0.011741986853059402
Input edges: 95
Output edg

Мы видим, что значения выдачи как для оценки хабовости так и авторитетности очень сильно отличаются от PageRank. Я думаю, что это достигается за счет того, что при вычислении hits оценок используются как входящие, так и выходящие дуги (правда в зависимости от оценки, что-то используется при вычислениях для самой вершины, а что-то для соседей); в то же время для pagerank учитываются только входящие вершины.


Говоря непосредственно про значения хабовости и авторитетности, можно заметить, что порядок этих оценок похож. Также, можно заметить, что топ-10 страниц для обеих оценок состоит из очень похожих ссылок на списки фильм или американских фильмов разных годов. Похоже, что они образуют некий связный подграф и как бы кучкуются вместе, имеют очень много ссылок друг на другу. Отсюда кажется, что получается рост как значения хабовости так и авторитетности внутри этой группы. Кстати, даже в топ-100 лишь ссылки на списки фильмов разных годов.

Заметим, что ни в одном топе нет ссылки на https://en.wikipedia.org/wiki/United_States, которая есть в топе для PageRank. Наверное это потому, что у ее соседей малая хабовость. Поэтому несмотря на большое количество входящих ссылок, значение авторитетности растет медленнее, чем у списков фильмов.

Я также проверил, что все ссылки из топа по хабовости ссылаются на все ссылки из топа по авторитетности. Наоборот примерно в 50% случаев. Т.о. кажется, что подтверждается предположение о том, что засчет близкого (в терминах достижимости) расположения этих ссылок относительно друг друга, происходит быстрое накопление ими хабовости и авторитетности.

Вообще ситуация кажется чем-то похожей на ту, которая случается с PageRank при большом alpha. Т.е. растет роль циклов. С поправкой на то, что это другой алгоритм в принципе и поэтому топы в принципе различаются.

Возможно также, что мы столкнулись с ситуацией описанной в https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC_HITS#%D0%9D%D0%B5%D0%B4%D0%BE%D1%81%D1%82%D0%B0%D1%82%D0%BA%D0%B8_HITS пример про Jaguar. Т.е. алгоритм "сместил" на в сторону американских фильмов, хотя изначально мы якобы искали США (мы конечно ничего не искали, но кажется так).